# <div align="center">YOLOv5 model 🤖</div>
<hr>

### In this notebook, you can train and make predictions using YOLOv5 on my dataset. 🚗🅿️

In [1]:
import sys
import os
import shutil
import torch
import sys
import subprocess

# Add the path to the utils folder to the system path
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.append('../')

from utils.functions import is_custom_model

repo_path = os.getcwd()
repo_path = os.path.dirname(repo_path)

yolov5_path = os.path.join(repo_path, 'yolov5')

# Check if GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
device_str = str(device)

In [3]:
!git clone https://github.com/ultralytics/yolov5.git "{yolov5_path}"
!python "{repo_path}/utils/yolov5start.py" --reporoot "{repo_path}"

fatal: destination path 'c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot\yolov5' already exists and is not an empty directory.


Repo path: c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot
Yolo path: c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot\yolov5
Data copied to c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot\yolov5\data/custom-data ✅
Only car labels left ✅
Only car labels left ✅
Dataset split into train and val sets ✅
Data augmentation done ✅
Data augmentation done ✅


In [4]:
# # notebook git clone yolov5
# !git clone https://github.com/ultralytics/yolov5.git $yolov5_path

# # copy data to yolov5/data/custom-data
# !python $repo_path/utils/yolov5start.py --reporoot $repo_path

In [5]:
%cd $yolov5_path

%pip install -r requirements.txt

c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot\yolov5


c:\Users\Evaldo Raynardi\anaconda3\envs\machineLearning\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Note: you may need to restart the kernel to use updated packages.


In [6]:
def yolov5_train(model, device="cpu", fold=0):
    model, model_path = is_custom_model(model, "5")

    !python train.py \
        --weights "{model_path}" \
        --img 1920 \
        --data "{yolov5_path}/data/custom-data/fold_{fold}/dataset.yaml" \
        --epochs 100 \
        --batch 6 \
        --project "{repo_path}/models/{model[:-3]}" \
        --device "{device}"

In [7]:
def yolov5_detect(model, device="cpu", conf=0.25, iou=0.45):
    model, model_path = is_custom_model(model, "5")
    
    # Remove folder if it exists
    if os.path.exists(os.path.join(repo_path, f'results/{model[:-3]}/')):
        shutil.rmtree(os.path.join(repo_path, f'results/{model[:-3]}/'))
    

    # 
    !python detect.py \
        --img 1920 \
        --source "{repo_path}/data/images" \
        --weights "{model_path}" \
        --conf {conf} \
        --iou {iou} \
        --save-txt \
        --project "{repo_path}/results" \
        --name "{model[:-3]}/yolo_images" \
        --device "{device}"

    # Change labels directory
    shutil.move(os.path.join(repo_path, f'results/{model[:-3]}/yolo_images/labels'), os.path.join(repo_path, f'results/{model[:-3]}/labels'))

In [8]:
def yolov5_trains(model, device="cpu", fold=0, resume=False):
    model, model_path = is_custom_model(model, "5")
    data_path = f"{yolov5_path}/data/custom-data/fold_{fold}/dataset.yaml"
    project_path = f"{repo_path}/models/{model[:-3]}"
    log_file = f"{project_path}/training_log.txt"

    # Path resume model
    resume_path = os.path.join(project_path, "weights", "last.pt")

    if resume:
        command = ["python", "train.py", "--resume", resume_path]
    else:
        command = [
            "python", "train.py",
            "--weights", model_path,
            "--img", "1920",
            "--data", data_path,
            "--epochs", "100",
            "--batch", "6",
            "--project", project_path,
            "--device", device
        ]

    with open(log_file, "w") as f:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        for line in process.stdout:
            print(line, end='')  # tampilkan ke terminal
            f.write(line)     

# <div align="center">Train your own YOLOv5 model on my dataset 🤖🚗🅿️</div>
<hr>

## Getting Started

To get started, simply call the `yolov5_train(model, device)` function. The `device` parameter specifies what device you want to use (cuda device, i.e. 0 or 0,1,2,3 or cpu), the `model` parameter specifies the model configuration you want to use and the `fold` parameter specifies the fold you want to use in training. There are three options available:

### 1. Standard YOLOv5 Model
You can use one of the pre-trained standard YOLOv5 models by providing the model name. For example:
```python
yolov5_train('yolov5n')
```
This will use the 'yolov5n' model for object detection and training. For more information on the available standard YOLOv5 models, you can refer to the [YOLOv5 models](https://github.com/ultralytics/yolov5#pretrained-checkpoints) repository.

### 2. Custom Model
If you want to train a custom model, you can provide the path to the model weights file. For example:
```python
yolov5_train(f'{repo_path}/models/yolov5n/exp/weights/best.pt')
```
This will allow you to train your own YOLOv5 model using the specified weights.

### 3. Uploaded Model
If you manually added a model to the models folder and want to train using that model, you can provide the name of the model file. For example:
```python
yolov5_train('my_yolov5n.pt')
```
This will use the 'my_yolov5n.pt' model for object detection and training.

## Customizing Training Arguments

If you want to customize the training arguments, you can modify the `yolov5_train(model)` function. The function takes care of the object detection and training process, but you can explore and modify the code inside the function according to your specific needs.
<hr>

#### If you want to train, just uncomment the following lines

In [9]:
# name:
# yolov5_train('yolov5n.pt', device, fold=0)
# yolov5_train('yolov5n.pt', device, fold=1)
yolov5_trains("yolov5n.pt", device_str, fold=0, resume=False)
# yolov5_trains("yolov5n.pt", device_str, fold=0, resume=True)

# path:
# yolov5_train(f'{repo_path}/models/yolov5n/exp/weights/best.pt', device)
yolov5_trains(f'{repo_path}/models/yolov5n/exp/weights/best.pt', device_str, resume=False)
# yolov5_trains(f'{repo_path}/models/yolov5n/exp/weights/best.pt', device_str, resume=True)

# uploaded model:
#yolov5_train('my_yolov5n.pt', device, fold=0)
#yolov5_train('my_yolov5n.pt', device, fold=1)

train: weights=yolov5n.pt, cfg=, data=c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot\yolov5/data/custom-data/fold_0/dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=100, batch_size=6, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot/models/yolov5n, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot 

# <div align="center">Make predictions with your own YOLOv5 model on my dataset 🔮🚗🔍</div>
<hr>

## Getting Started

To get started, simply call the `yolov5_detect(model, device)` function. The `device` parameter specifies what device you want to use (cuda device, i.e. 0 or 0,1,2,3 or cpu), the `model` parameter specifies the model configuration you want to use. There are three options available:

### 1. Standard YOLOv5 Model
You can use one of the pre-trained standard YOLOv5 models by providing the model name. For example:
```python
yolov5_detect('yolov5n')
```
This will use the 'yolov5n' model for object detection and prediction. For more information on the available standard YOLOv5 models, you can refer to the [YOLOv8 models](https://github.com/ultralytics/yolov5#pretrained-checkpoints) repository.

### 2. Custom Model
If you want to make predictions with a custom model, you can provide the path to the model weights file. For example:
```python
yolov5_detect(f'{repo_path}/models/yolov5n/exp/weights/best.pt')
```
This will allow you to make predictions on my dataset using the specified weights.

### 3. Uploaded Model
If you manually added a model to the models folder and want to make predictions using that model, you can provide the name of the model file. For example:
```python
yolov5_detect('my_yolov5n.pt')
```
This will use the 'my_yolov5n.pt' model for object prediction.

## Customizing Prediction Arguments

If you want to customize the prediction arguments, you can modify the `yolov5_detect(model)` function. The function takes care of the object detection and prediction process, but you can explore and modify the code inside the function according to your specific needs.
<hr>

#### If you want to make predictions, just uncomment the following lines

In [ ]:
# name:
yolov5_detect('yolov5n.pt', device)

# path:
yolov5_detect(f'{repo_path}/models/yolov5n/exp/weights/best.pt', device)

# uploaded model:
# yolov5_detect('yolov5n_fold_0.pt', device, conf=0.4, iou=0.35)

detect: weights=['yolov5n.pt'], source=c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot/data/images, data=data\coco128.yaml, imgsz=[1920, 1920], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=cpu, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=c:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\testing\YOLO-Parking-Spot/results, name=yolov5n/yolo_images, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'C:\Users\Evaldo': No such file or directory
YOLOv5  2025-6-5 Python-3.12.7 torch-2.7.0+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
image 1/23 C:\Users\Evaldo Raynardi\OneDrive\Desktop\Kuilah\Semester 4\Machine Learning\